<a href="https://colab.research.google.com/github/danilotpnta/UN-General-Debate-Analysis-SDGs/blob/main/notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# UN-General-Debate-Analysis-SDGs

This project analyzes the UN General Debate Corpus from 1970 to 2023. It includes exploratory data analysis (EDA), predictive modeling, and data visualizations focusing on uncovering insights from political speeches and their connection to global challenges.

The project is divided into the following sections:

1. Data Preprocessing
2. Exploratory Data Analysis
3. Predictive Modeling
4. Data Visualization

## If running from Google Colab

In [19]:
# Clone your GitHub repository
!git clone https://github.com/danilotpnta/UN-General-Debate-Analysis-SDGs.git

# Navigate to the repository folder
%cd /content/UN-General-Debate-Analysis-SDGs

Cloning into 'UN-General-Debate-Analysis-SDGs'...
remote: Enumerating objects: 34, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 34 (delta 12), reused 22 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (34/34), 77.71 KiB | 5.55 MiB/s, done.
Resolving deltas: 100% (12/12), done.
[Errno 2] No such file or directory: '/content/UN-General-Debate-Analysis-SDGs'
/Users/datoapanta/code/UN-General-Debate-Analysis-SDGs


## Data Preprocessing

#### Donwloading the dataset

In [21]:
from importlib import reload
import utils.dataverse_downloader as dataverse_downloader

# Reload the module
reload(dataverse_downloader)

# Call the function to download all files
dataverse_downloader.download_dataset()

File already downloaded: Raw_PDFs_1946-1969.tgz
File already downloaded: Raw_PDFs_1970-1990.tgz
File already downloaded: Raw_PDFs_1991-2022.tgz
File already downloaded: README.txt
File already downloaded: Speakers_by_session.xlsx
File already downloaded: UNGDC_1946-2023.tgz
Uncompressing data...
Data already uncompressed: data/raw/Raw_PDFs_1946-1969
Data already uncompressed: data/raw/Raw_PDFs_1970-1990
Data already uncompressed: data/raw/Raw_PDFs_1991-2022
Data already uncompressed: data/raw/UNGDC_1946-2023


In [ ]:
# Load the data
import pandas as pd

# import tgz file into a dataframe
df = pd.read_csv('data/UN-general-debates-1970-2016-Part1.csv')



## Exploratory Data Analysis
